In [175]:
import numpy as np
from scipy.sparse import coo_matrix, vstack
from scipy.sparse.linalg import cg
import mcubes
import open3d as o3d
import trimesh

In [176]:
def generate_point_cloud_with_normals(mesh, num_points=1000):
    # Randomly sample points from the mesh
    random_indices = np.random.choice(mesh.vertices.shape[0], num_points, replace=False)
    points = mesh.vertices[random_indices]
    normals = mesh.vertex_normals[random_indices]
    pc_mesh = trimesh.Trimesh(vertices=points, vertex_normals=normals)
    return pc_mesh

def save_point_cloud(mesh, mesh_file_name):
    # Save the point cloud as numpy array file, because Trimesh cannot preserve normals when saving point cloud 
    point_cloud_filename = "PointCloud_" + mesh_file_name.split('.')[0]
    np.savez(point_cloud_filename, vertices=mesh.vertices, vertex_normals=mesh.vertex_normals)

def load_point_cloud(mesh_file_name):
    # Load the point cloud from numpy array file
    input_point_cloud_filename = "PointCloud_" + mesh_file_name.split('.')[0] + ".npz"
    data = np.load(input_point_cloud_filename)
    return trimesh.Trimesh(vertices=data['vertices'], vertex_normals=data['vertex_normals'])

In [177]:
mesh_file_name = "bun_zipper_res3.ply"
sample_num_points = 1000

if True:
    # Generate point cloud from mesh
    original_mesh = trimesh.load_mesh(mesh_file_name)
    point_cloud = generate_point_cloud_with_normals(original_mesh, num_points=sample_num_points)
    save_point_cloud(point_cloud, mesh_file_name)

In [178]:
input_point_cloud = load_point_cloud(mesh_file_name)
num_cubes = np.array([64, 64, 64])
padding = 4

input_points = input_point_cloud.vertices
input_normals = -input_point_cloud.vertex_normals # Reverse the normals because they are pointing inwards

def get_voxel_grid(vertices, num_cubes, padding):
    bounding_box = np.max(vertices, axis=0) - np.min(vertices, axis=0)
    voxel_size = np.max(bounding_box) / num_cubes
    origin = np.min(vertices, axis=0) - padding * voxel_size
    voxel_resolution = num_cubes + 2 * padding
    return origin, voxel_size, voxel_resolution

origin, voxel_size, voxel_resolution = get_voxel_grid(input_points, num_cubes, padding)

print(origin)
print(voxel_size)
print(voxel_resolution)

[-0.10344884  0.02391646 -0.07125964]
[0.00239689 0.00239689 0.00239689]
[72 72 72]


In [179]:
def fd_grad(n_cube, s_cube):
    # TODO
    nx, ny, nz = n_cube
    hx, hy, hz = s_cube

    idx_primary_grid = np.arange(nx * ny * nz).reshape((nx, ny, nz))

    num_staggered_grid_x = (nx - 1) * ny * nz
    num_staggered_grid_y = nx * (ny - 1) * nz
    num_staggered_grid_z = nx * ny * (nz - 1)
    idx_col_x = np.concatenate((idx_primary_grid[1:, ...].flatten(), idx_primary_grid[:-1, :, :].flatten()))
    idx_col_y = np.concatenate((idx_primary_grid[:, 1:, :].flatten(), idx_primary_grid[:, :-1, :].flatten()))
    idx_col_z = np.concatenate((idx_primary_grid[:, :, 1:].flatten(), idx_primary_grid[:, :, :-1].flatten()))
    row_idx_x = np.arange(num_staggered_grid_x)
    row_idx_y = np.arange(num_staggered_grid_y)
    row_idx_z = np.arange(num_staggered_grid_z)
    row_idx_x = np.tile(row_idx_x, 2)
    row_idx_y = np.tile(row_idx_y, 2)
    row_idx_z = np.tile(row_idx_z, 2)

    data_term_x = [1 / hx] * num_staggered_grid_x + [-1 / hx] * num_staggered_grid_x
    data_term_y = [1 / hy] * num_staggered_grid_y + [-1 / hy] * num_staggered_grid_y
    data_term_z = [1 / hz] * num_staggered_grid_z + [-1 / hz] * num_staggered_grid_z

    Dx = coo_matrix((data_term_x, (row_idx_x, idx_col_x)), shape=(num_staggered_grid_x, nx * ny * nz))
    Dy = coo_matrix((data_term_y, (row_idx_y, idx_col_y)), shape=(num_staggered_grid_y, nx * ny * nz))
    Dz = coo_matrix((data_term_z, (row_idx_z, idx_col_z)), shape=(num_staggered_grid_z, nx * ny * nz))
    G = vstack((Dx, Dy, Dz))

    return G

G = fd_grad(voxel_resolution, voxel_size)

In [180]:
def trilinear_interpolation_weights(n_cube, corner, points, s_cube, direction=None):
    # TODO
    nx, ny, nz = n_cube
    hx, hy, hz = s_cube

    if direction == "x":  # center of the corner cube
        corner[0] += 0.5 * hx
    elif direction == "y":
        corner[1] += 0.5 * hy
    elif direction == "z":
        corner[2] += 0.5 * hz
    else:
        pass
    # grid coordinates / indices
    # minusing corner's coordinates makes the samples started from origin and the floor of them are the cubes' index
    # they are in. the auther call it as "grid coordinates/indices"

    x0 = np.floor((points[:, 0] - corner[0]) / hx).astype(int)  # (N, )
    y0 = np.floor((points[:, 1] - corner[1]) / hy).astype(int)  # (N, )
    z0 = np.floor((points[:, 2] - corner[2]) / hz).astype(int)  # (N, )
    x1 = x0 + 1  # (N, )
    y1 = y0 + 1  # (N, )
    z1 = z0 + 1  # (N, )

    # the coordinates of samples in their local cubes, the numeral values of them are the percentages along each axis.
    xd = (points[:, 0] - corner[0]) / hx - x0  # (N, )
    yd = (points[:, 1] - corner[1]) / hy - y0  # (N, )
    zd = (points[:, 2] - corner[2]) / hz - z0  # (N, )

    # data terms for the trilinear interpolation weight matrix
    weight_000 = (1 - xd) * (1 - yd) * (1 - zd)
    weight_100 = xd * (1 - yd) * (1 - zd)
    weight_010 = (1 - xd) * yd * (1 - zd)
    weight_110 = xd * yd * (1 - zd)
    weight_001 = (1 - xd) * (1 - yd) * zd
    weight_101 = xd * (1 - yd) * zd
    weight_011 = (1 - xd) * yd * zd
    weight_111 = xd * yd * zd
    data_term = np.concatenate((weight_000,
                                weight_100,
                                weight_010,
                                weight_110,
                                weight_001,
                                weight_101,
                                weight_011,
                                weight_111))

    row_idx = np.arange(points.shape[0])
    row_idx = np.tile(row_idx, 8)

    if direction == "x":
        num_grids = (nx - 1) * ny * nz
        staggered_grid_idx = np.arange((nx - 1) * ny * nz).reshape((nx - 1, ny, nz))
    elif direction == "y":
        num_grids = nx * (ny - 1) * nz
        staggered_grid_idx = np.arange(nx * (ny - 1) * nz).reshape((nx, ny - 1, nz))
    elif direction == "z":
        num_grids = nx * ny * (nz - 1)
        staggered_grid_idx = np.arange(nx * ny * (nz - 1)).reshape((nx, ny, nz - 1))
    else:
        num_grids = nx * ny * nz
        staggered_grid_idx = np.arange(nx * ny * nz).reshape((nx, ny, nz))

    col_idx_000 = staggered_grid_idx[x0, y0, z0]
    col_idx_100 = staggered_grid_idx[x1, y0, z0]
    col_idx_010 = staggered_grid_idx[x0, y1, z0]
    col_idx_110 = staggered_grid_idx[x1, y1, z0]
    col_idx_001 = staggered_grid_idx[x0, y0, z1]
    col_idx_101 = staggered_grid_idx[x1, y0, z1]
    col_idx_011 = staggered_grid_idx[x0, y1, z1]
    col_idx_111 = staggered_grid_idx[x1, y1, z1]
    col_idx = np.concatenate((col_idx_000,
                                col_idx_100,
                                col_idx_010,
                                col_idx_110,
                                col_idx_001,
                                col_idx_101,
                                col_idx_011,
                                col_idx_111))

    W = coo_matrix((data_term, (row_idx, col_idx)), shape=(points.shape[0], num_grids))
    return W

print(voxel_resolution, origin, input_points, voxel_size)
Wx = trilinear_interpolation_weights(voxel_resolution, origin, input_points, voxel_size, direction="x")
Wy = trilinear_interpolation_weights(voxel_resolution, origin, input_points, voxel_size, direction="y")
Wz = trilinear_interpolation_weights(voxel_resolution, origin, input_points, voxel_size, direction="z")
W = trilinear_interpolation_weights(voxel_resolution, origin, input_points, voxel_size)

[72 72 72] [-0.10344884  0.02391646 -0.07125964] [[-0.0268168   0.123904    0.0208113 ]
 [ 0.00267879  0.038424   -0.00319748]
 [-0.0577616   0.0644884  -0.00779097]
 ...
 [ 0.0473431   0.0499217   0.0295077 ]
 [ 0.0318737   0.0877439  -0.0192705 ]
 [-0.0844667   0.0817669   0.00249573]] [0.00239689 0.00239689 0.00239689]


In [181]:
V = np.concatenate(
    [
        Wx.T @ input_normals[:, 0],
        Wy.T @ input_normals[:, 1],
        Wz.T @ input_normals[:, 2]
    ]
)
np.sum(V) / V.shape[0]

-8.568130194102954e-06

In [182]:
g, _ = cg(G.T @ G, G.T @ V, maxiter=2000, tol=1e-5)
g -= np.mean(W @ g)
g_field = g.reshape(voxel_resolution)
vertices, triangles = mcubes.marching_cubes(g_field, 0)

In [183]:
vertices[:, 0] = vertices[:, 0] * voxel_size[0] + origin[0]
vertices[:, 1] = vertices[:, 1] * voxel_size[1] + origin[1]
vertices[:, 2] = vertices[:, 2] * voxel_size[2] + origin[2]

output_mesh = trimesh.Trimesh(vertices=vertices, faces=triangles)
output_mesh.export("output_mesh.ply")

b'ply\nformat binary_little_endian 1.0\ncomment https://github.com/mikedh/trimesh\nelement vertex 14238\nproperty float x\nproperty float y\nproperty float z\nelement face 28472\nproperty list uchar int vertex_indices\nend_header\n\xe0h\xbe\xbd\xc8\xc9\xf7=\x1c3\xac<\x06\xc6\xbd\xbdw%\xf7=\x1c3\xac<\x06\xc6\xbd\xbd\xc8\xc9\xf7=\xf9<\x99<\x06\xc6\xbd\xbd\xc8\xc9\xf7=\xa8;\xb7<\x84\xca\xbd\xbdq\xb2\xfc=\xd7\xed\x84<\x06\xc6\xbd\xbd\xa6\x85\xfc=\xd7\xed\x84<\x06\xc6\xbd\xbdq\xb2\xfc=\xe9\xa6\x84<\x80\x96\xbe\xbdq\xb2\xfc=y\x90\x98<\x06\xc6\xbd\xbd\xba\xf9\xf7=y\x90\x98<\xbb1\xbe\xbdq\xb2\xfc=\x1c3\xac<\x06\xc6\xbd\xbdq\xb2\xfc=K\xc9\xb7<\x06\xc6\xbd\xbd\xd1\xd8\xfc=\xd7\xed\x84<\xf3\x06\xbe\xbd\x8d\xcd\x00>y\x90\x98<\x06\xc6\xbd\xbd\x8d\xcd\x00>\xbf\x15\x93<\x06\xc6\xbd\xbd\xa79\x00>\x1c3\xac<\x06\xc6\xbd\xbd\x8d\xcd\x00>\x98\xef\xa6<\x06\xc6\xbd\xbd{\x86\x01>y\x90\x98<\xd3\x8a\xb9\xbdw\xf8\xed=%Q;<]\xdd\xb8\xbd\x8f\x03\xed=%Q;<]\xdd\xb8\xbdw\xf8\xed=*\xd70<\x10y\xba\xbdw\xf8\xed=j\x96b<]